In [ ]:
!pip install somoclu scikit-learn tensorflow numpy matplotlib opencv-python

import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import somoclu  # SOM library

In [ ]:
# Dataset Path
data_dir = "/kaggle/input/swedishleaf/SwedishLeaf"
IMG_SIZE = (64, 64)  # Use a smaller size for efficient SOM training
NUM_CLASSES = 10

In [ ]:
# Function to Load and Preprocess Dataset
def load_images(data_dir, img_size):
    images, labels = [], []
    class_names = sorted(os.listdir(data_dir))  # Class folders
    
    for label, class_name in enumerate(class_names):
        class_dir = os.path.join(data_dir, class_name)
        if not os.path.isdir(class_dir):
            continue
        for img_file in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_file)
            try:
                img = cv2.imread(img_path)
                img = cv2.resize(img, img_size)  # Resize to consistent size
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale for SOM
                images.append(img.flatten())  # Flatten image to 1D vector
                labels.append(label)
            except Exception as e:
                print(f"Error loading {img_path}: {e}")

    return np.array(images, dtype="float32"), np.array(labels, dtype="int")

In [ ]:
# Load dataset
x_data, y_data = load_images(data_dir, IMG_SIZE)

# Normalize pixel values
x_data /= 255.0

# Split into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42, stratify=y_data)

print(f"Train data: {x_train.shape}, Test data: {x_test.shape}")


In [ ]:
# Define SOM grid size
som_shape = (20, 20)  # 20x20 SOM grid

# Initialize and train SOM
som = somoclu.Somoclu(som_shape[0], som_shape[1], compactsupport=False)
som.train(x_train, epochs=100, radius0=5, scale0=0.01)  # Carefully tuned parameters

In [ ]:
# Visualize SOM feature map
plt.figure(figsize=(10, 10))
plt.imshow(som.umatrix, cmap="coolwarm")
plt.colorbar()
plt.title("SOM Feature Map")
plt.show()


In [ ]:
# Extract BMU (Best Matching Unit) features
bmus_train = np.array([som.bmus[i][0] * som_shape[1] + som.bmus[i][1] for i in range(len(x_train))])
bmus_test = np.array([som.bmus[i][0] * som_shape[1] + som.bmus[i][1] for i in range(len(x_test))])

# Convert BMU positions to one-hot encoding
bmus_train_one_hot = np.eye(som_shape[0] * som_shape[1])[bmus_train]
bmus_test_one_hot = np.eye(som_shape[0] * som_shape[1])[bmus_test]

print(f"SOM Features (Train): {bmus_train_one_hot.shape}")
print(f"SOM Features (Test): {bmus_test_one_hot.shape}")


In [ ]:
# Train an SVM classifier on the extracted SOM features
classifier = SVC(kernel='rbf', C=10, gamma='scale')  # Tuned parameters
classifier.fit(bmus_train_one_hot, y_train)

# Predict and evaluate
y_pred = classifier.predict(bmus_test_one_hot)
accuracy = accuracy_score(y_test, y_pred)

print(f"\nClassification Accuracy: {accuracy * 100:.2f}%")
